In [6]:
# Install required libraries
!pip install transformers torch accelerate

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

In [2]:
model_name = "unsloth/gemma-2b-it"  # Replace with "gemma-7b" for the 7B version
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.float16)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/40.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/724 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/5.01G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/154 [00:00<?, ?B/s]

In [23]:
# Define a function for question answering
def ask_question(question, context=None):
    # Format the input prompt
    if context:
        prompt = f"Context: {context}\nQuestion: {question}\nAnswer:"
    else:
        prompt = f"Question: {question}\nAnswer:"

    # Tokenize the input
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    # Generate the answer
    with torch.no_grad():
        outputs = model.generate(
            inputs["input_ids"],
            max_new_tokens=2048,  # Adjust based on how long you want the answer to be
            do_sample=True,      # Enable sampling for more creative answers
            temperature=0.7,    # Adjust for randomness (lower = more deterministic)
            top_p=0.9,          # Nucleus sampling parameter
        )

    # Decode the generated text
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract only the answer part (remove the prompt)
    answer = answer.split("Answer:")[-1].strip()
    return answer

In [4]:
# Move model to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (up_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (down_proj): Linear(in_features=16384, out_features=2048, bias=False)
          (act_fn): GELUActivation()
        )
        (input_layernorm): GemmaRMSNorm((2048,), eps=1e-06)
        (post_attention_layernorm): GemmaRMSNorm((2048,), eps=1e-06)
      )
    )
    (norm): GemmaRMSNorm((2048,), 

In [5]:
context = """
Ты умный чат-бот, который общается с обычным пользователем. Общайся с ним только на русском языке.
"""

In [18]:
question = "Расскажи о сфере анализа данных и о сфере искусственного интеллекта"

In [24]:
answer = ask_question(question, context)
print(f"Question: {question}")
print(f"Answer: {answer}")

Question: Расскажи о сфере анализа данных и о сфере искусственного интеллекта
Answer: **Сфера анализа данных**

* Определение: Часть исследования, которая посвящается обработке данных, анализум и выведению из них meaningful insights.
* Цели:
    * Выявление закономерностей и трендов в данных.
    * Разработка алгоритмов машинного обучения.
    * Анализ данных для выявления скрытых закономерностей.

**Сфера искусственного интеллекта**

* Определение: Часть AI, которая посвящается созданию искусственного интеллекта (AI).
* Цели:
    * Разработка алгоритмов машинного обучения.
    * Выявление скрытых закономерностей в данных.
    * Роль AI в различных областях, таких как бизнес, здравоохранение, образование и наука.
